In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/fangzhou/data/level2_i_20221202.csv')
df.head()

long = 200
short = 100


a = df['midp'].diff(long) / df['midp'].shift(long)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['long return'] = a

b = df['midp'].diff(short) / df['midp'].shift(short)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['short return'] = b

# only keep the factors
col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('long return')
col_reserved.append('short return')

new_df = df[col_reserved]
linear_df = new_df[1200:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)
linear_df

,fac1,fac2,fac3,fac4,fac5,fac6,fac7,fac8,fac9,fac10,...,fac26,fac27,fac28,fac29,fac30,fac31,fac32,fac33,long return,short return
0,0.000018,0.013011,0.999999,0.001871,-0.061324,-0.031032,-0.000112,-0.010945,0.002210,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000209,-0.000312,-0.000325,0.001302,0.000651
1,0.000019,0.001862,0.999990,0.001399,-0.060856,-0.030808,-0.000448,-0.010954,0.002212,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000209,-0.000312,-0.000325,0.001302,0.000651
2,0.000017,-0.005618,-0.999997,0.002332,-0.059300,-0.030743,-0.000449,-0.011102,0.002207,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000211,-0.000312,-0.000325,0.001302,0.000651
3,0.000018,0.001869,0.999990,0.002799,-0.056428,-0.030477,-0.000434,-0.011120,0.002209,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000211,-0.000312,-0.000325,0.001302,0.000651
4,0.000018,0.000000,0.000000,-0.000467,-0.053220,-0.030216,-0.000434,-0.011137,0.002211,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000211,-0.000312,-0.000325,0.001302,0.000651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72994,-0.000033,-0.005181,-0.999997,-0.002582,-0.008557,-0.007308,-0.011443,-0.020865,0.001662,-8.0,...,-0.6,0.000000,0.000219,-0.001025,-0.001025,-0.000104,0.000170,0.000316,-0.000633,-0.000633
72995,-0.000032,0.003442,0.999995,-0.001291,-0.008718,-0.007164,-0.011714,-0.020276,0.001666,-8.0,...,-0.6,0.000000,0.000219,-0.001019,-0.001019,-0.000102,0.000169,0.000316,-0.000633,-0.000633
72996,-0.000033,-0.003466,-0.499999,-0.001725,-0.008621,-0.007245,-0.011738,-0.019254,0.001663,-8.0,...,-0.6,0.000000,0.000219,-0.001011,-0.001011,-0.000100,0.000164,0.000226,-0.000633,-0.000633
72997,-0.000033,0.000000,0.000000,-0.001296,-0.008697,-0.007193,-0.012011,-0.018918,0.001660,-8.0,...,-0.6,0.000000,0.000219,-0.001010,-0.001010,-0.000096,0.000164,0.000226,-0.000633,-0.000633


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

window_size = 200

predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

#limit = len(linear_df)
limit = 20000

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i, :-2]
    train_y_long = linear_df.iloc[i - window_size : i, -2]
    train_y_short = linear_df.iloc[i - window_size : i, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    model_short = RandomForestRegressor(n_estimators = 15, max_depth = 5, random_state = 11)
    model_long = RandomForestRegressor(n_estimators = 15, max_depth = 5, random_state = 11)

    model_short.fit(train_X_short, train_y_short)
    model_long.fit(train_X_long, train_y_long)

    predict_short.append(model_short.predict(test_X))
    predict_long.append(model_long.predict(test_X))

    R_square_short.append(model_short.score(validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(validate_X_long, validate_y_long)))

i = 1000
R_square_short = 0.9630571514797991
R_square_long = 0.9927871938354067
i = 2000
R_square_short = 1.0
R_square_long = 0.8768031684027777
i = 3000
R_square_short = 0.9823196970759228
R_square_long = 0.8449797988860706
i = 4000
R_square_short = 0.9876596626136935
R_square_long = 0.9460695458870312
i = 5000
R_square_short = 0.9990713919159028
R_square_long = 0.988563678909058
i = 6000
R_square_short = 0.8365505602340683
R_square_long = 0.8950544949382732
i = 7000
R_square_short = 0.8258186688878524
R_square_long = 0.9077249657535797
i = 8000
R_square_short = 0.9986523483474511
R_square_long = 0.9557893880552204
i = 9000
R_square_short = 0.9615714783252561
R_square_long = 0.6739358054271817
i = 10000
R_square_short = 0.971666670492118
R_square_long = 0.9998367045520744
i = 11000
R_square_short = 0.9969832612026139
R_square_long = 0.9692174076970175
i = 12000
R_square_short = 0.966652192792181
R_square_long = 0.9990740724991172
i = 13000
R_square_short = 0.9379125233413256
R_square_

In [8]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)
new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df

,predict_short,predict_long,R_square_short,R_square_long,long return,short return,predict_long_sign,predict_short_sign
0,-0.000607,5.636856e-04,0.877888,0.865523,0.0,-0.00065,1,-1
1,-0.000563,4.389488e-04,0.986059,0.905499,0.0,-0.00065,1,-1
2,-0.000563,2.601626e-04,0.796831,0.873444,0.0,-0.00065,1,-1
3,-0.000650,2.601626e-04,0.987529,0.810858,0.0,-0.00065,1,-1
4,-0.000650,1.734417e-04,0.970788,0.760091,0.0,-0.00065,1,-1
...,...,...,...,...,...,...,...,...
19795,0.000000,0.000000e+00,0.659510,0.616022,0.0,0.00000,0,0
19796,0.000000,-8.715018e-05,0.977778,0.820485,0.0,0.00000,-1,0
19797,0.000000,-1.300813e-04,0.441975,0.849207,0.0,0.00000,-1,0
19798,0.000000,1.322695e-05,0.982571,0.894584,0.0,0.00000,1,0


In [9]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())

print(new_df[['predict_short', 'short return']].corr())
print(new_df[['predict_long', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.775928
short return                  0.775928      1.000000
                   predict_long_sign  long return
predict_long_sign             1.0000       0.8097
long return                   0.8097       1.0000
               predict_short  short return
predict_short       1.000000      0.960699
short return        0.960699      1.000000
              predict_long  long return
predict_long      1.000000     0.979667
long return       0.979667     1.000000


In [10]:
predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i, :-2]
    train_y_long = linear_df.iloc[i - window_size : i, -2]
    train_y_short = linear_df.iloc[i - window_size : i, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    model_short = GradientBoostingRegressor(n_estimators = 15, max_depth = 3, learning_rate = 0.1)
    model_long = GradientBoostingRegressor(n_estimators = 15, max_depth = 3, learning_rate = 0.1)

    model_short.fit(train_X_short, train_y_short)
    model_long.fit(train_X_long, train_y_long)

    predict_short.append(model_short.predict(test_X))
    predict_long.append(model_long.predict(test_X))

    R_square_short.append(model_short.score(validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(validate_X_long, validate_y_long)))

i = 1000
R_square_short = 0.9219935523803773
R_square_long = 0.9325236852543799
i = 2000
R_square_short = 1.0
R_square_long = 0.847940356603736
i = 3000
R_square_short = 0.926010441861744
R_square_long = 0.7772339140712318
i = 4000
R_square_short = 0.9280793586159781
R_square_long = 0.9066759002006501
i = 5000
R_square_short = 0.9410273508976668
R_square_long = 0.944953111903187
i = 6000
R_square_short = 0.8085303335214228
R_square_long = 0.8600091431147483
i = 7000
R_square_short = 0.6852795689262589
R_square_long = 0.8209032272375572
i = 8000
R_square_short = 0.9121612200474857
R_square_long = 0.9307164336892191
i = 9000
R_square_short = 0.9371245118312215
R_square_long = 0.6026882860860123
i = 10000
R_square_short = 0.9580094712007914
R_square_long = 0.953291480082066
i = 11000
R_square_short = 0.9487257823913573
R_square_long = 0.8941376175522849
i = 12000
R_square_short = 0.9003206752970133
R_square_long = 0.956425305007506
i = 13000
R_square_short = 0.9049766225494357
R_square_lo

In [11]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)

new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df

,predict_short,predict_long,R_square_short,R_square_long,long return,short return,predict_long_sign,predict_short_sign
0,-0.000465,0.000606,0.825431,0.805176,0.0,-0.00065,1,-1
1,-0.000465,0.000402,0.928193,0.816790,0.0,-0.00065,1,-1
2,-0.000468,0.000161,0.881838,0.751907,0.0,-0.00065,1,-1
3,-0.000469,0.000246,0.944459,0.733594,0.0,-0.00065,1,-1
4,-0.000471,0.000078,0.944812,0.807054,0.0,-0.00065,1,-1
...,...,...,...,...,...,...,...,...
19795,-0.000029,-0.000006,0.579919,0.621578,0.0,0.00000,-1,-1
19796,-0.000026,0.000024,0.936162,0.822260,0.0,0.00000,1,-1
19797,-0.000027,0.000005,0.566238,0.789914,0.0,0.00000,1,-1
19798,-0.000026,0.000024,0.911541,0.878007,0.0,0.00000,1,-1


In [12]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.647457
short return                  0.647457      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.710441
long return                 0.710441     1.000000
